In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path
import neptune # for logging progress and results
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import set_environment_variables, neptune_output_figures
neptune_kwargs = {'project':"commander.pho/PhoDibaLongShort2023",
'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from attrs import define, field, fields, Factory

# Pho's Formatting Preferences
# from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
# set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_file_only, matplotlib_configuration, matplotlib_configuration_update
from neuropy.core.neuron_identities import NeuronIdentityTable, neuronTypesList, neuronTypesEnum

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.neurons import NeuronType
from neuropy.core.user_annotations import UserAnnotationsManager
from neuropy.core.position import Position

from pyphocorehelpers.print_helpers import print_object_memory_usage, print_dataframe_memory_usage, print_value_overview_only, DocumentationFilePrinter, print_keys_if_possible, generate_html_string

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap
import pyphoplacecellanalysis.External.pyqtgraph as pg

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons
from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler # for `post_compute_validate(...)`
from neuropy.utils.mixins.AttrsClassHelpers import AttrsBasedClassHelperMixin, serialized_field, serialized_attribute_field, non_serialized_field
from neuropy.utils.mixins.HDF5_representable import HDF_DeserializationMixin, post_deserialize, HDF_SerializationMixin, HDFMixin
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BasePositionDecoder

# Plotting
import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()
# import pylustrator # call `pylustrator.start()` before creating your first figure in code.
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import DataSeriesColorHelpers
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import UnitColoringMode, DataSeriesColorHelpers
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _build_default_tick, build_scatter_plot_kwargs
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DScrollWindowPlot import Render2DScrollWindowPlotMixin, ScatterItemData
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_extended_programmatic_figures, batch_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends

from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3, main_complete_figure_generations

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL\TESTING\Logging\debug_com.PhoHale.Spike3D.pipeline.log


# Load Pipeline

In [2]:
%pdb off
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'
local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=self.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
# curr_context: IdentifyingContext = good_contexts_list[1] # select the session from all of the good sessions here.
curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15') # DONE. Very good. Many good Pfs, many good replays.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43') # DONE, might be the BEST SESSION, good example session with lots of place cells, clean replays, and clear bar graphs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31') # DONE, Good Pfs but no good replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6') # BAD, 2023-07-14, unsure why still.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19') # DONE, GREAT, both good Pfs and replays!
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25') # DONE, Added replay selections. Very "jumpy" between the starts and ends of the track.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40') # DONE, Added replay selections. A TON of putative replays in general, most bad, but some good.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46') # DONE, added replay selections. Very few (like 12) replays each.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3') # DONE, Good Pfs, no good epochs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30') # DONE, okay replays (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50') # DONE, very few replays (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54') # DONE, one replay each (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3') # BAD, Good Pfs strangely despite horrible map, no good epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59') # BAD, No Epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44') # DONE, good. Many good pfs, many good replays.
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0') # DONE, good?, replays selected, few
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25') # DONE, very few replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54') # DONE, replays selected, quite a few replays but few are very good.

local_session_parent_path: Path = local_session_root_parent_path.joinpath(curr_context.animal, curr_context.exper_name) # 'gor01', 'one' - probably not needed anymore
basedir: Path = local_session_parent_path.joinpath(curr_context.session_name).resolve()
print(f'basedir: {str(basedir)}')

# Read if possible:
saving_mode = PipelineSavingScheme.SKIP_SAVING
force_reload = False

# Force write:
# saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
# saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
# force_reload = True

## TODO: if loading is not possible, we need to change the `saving_mode` so that the new results are properly saved.


# ==================================================================================================================== #
# Load Pipeline                                                                                                        #
# ==================================================================================================================== #
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-full_session_LOO_decoding_analysis.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
# epoch_name_includelist = ['maze']
epoch_name_includelist = None
active_computation_functions_name_includelist=[#'_perform_estimated_epochs_computation',  # AL:WAYS OFF
                                            '_perform_baseline_placefield_computation',
                                        # '_perform_time_dependent_placefield_computation', # AL:WAYS OFF
                                        '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation' # AL:WAYS OFF
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding' # AL:WAYS OFF
                                    ]
# active_computation_functions_name_includelist=['_perform_baseline_placefield_computation']

curr_active_pipeline: NeuropyPipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_includelist=epoch_name_includelist,
                                        computation_functions_name_includelist=active_computation_functions_name_includelist,
                                        saving_mode=saving_mode, force_reload=force_reload,
                                        skip_extended_batch_computations=True, debug_print=False, fail_on_exception=False) # , active_pickle_filename = 'loadedSessPickle_withParameters.pkl'


if not force_reload: # not just force_reload, needs to recompute whenever the computation fails.
    try:
        curr_active_pipeline.load_pickled_global_computation_results()
    except Exception as e:
        print(f'cannot load global results: {e}')

# 6m 1.1s
# 12m 15.6s

try:
    ## Post Compute Validate 2023-05-16:
    was_updated = BatchSessionCompletionHandler.post_compute_validate(curr_active_pipeline) ## TODO: need to potentially re-save if was_updated
    if was_updated:
        try:
            curr_active_pipeline.save_pipeline(saving_mode=saving_mode)
        except Exception as e:
            ## TODO: catch/log saving error and indicate that it isn't saved.
            print(f'ERROR RE-SAVING PIPELINE after update. error: {e}')

    curr_active_pipeline.reload_default_computation_functions()
    extended_computations_include_includelist=['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding'] # do only specifiedl , 'long_short_rate_remapping'
    force_recompute_global = force_reload
    newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=force_recompute_global, debug_print=False)
    if (len(newly_computed_values) > 0) and (saving_mode.value != 'skip_saving'):
        print(f'newly_computed_values: {newly_computed_values}. Saving global results...')
        try:
            # curr_active_pipeline.global_computation_results.persist_time = datetime.now()
            # Try to write out the global computation function results:
            curr_active_pipeline.save_global_computation_results()
        except Exception as e:
            print(f'\n\n!!WARNING!!: saving the global results threw the exception: {e}')
            print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
    else:
        print(f'no changes in global results.')

except Exception as e:
    print(f'second half threw: {e}')

Automatic pdb calling has been turned OFF
basedir: W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15
Loading loaded session pickle file results : W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\loadedSessPickle.pkl... done.
Loading pickled pipeline success: W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\loadedSessPickle.pkl.
property already present in pickled version. No need to save.
using provided computation_functions_name_includelist: ['_perform_baseline_placefield_computation', '_perform_extended_statistics_computation', '_perform_position_decoding_computation', '_perform_firing_rate_trends_computation', '_perform_pf_find_ratemap_peaks_computation', '_perform_two_step_position_decoding_computation']
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is sm

# End Run

In [3]:
## long_short_decoding_analyses:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
## Extract variables from results object:
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global

# (long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_computation_config, short_computation_config, global_computation_config = [curr_active_pipeline.computation_results[an_epoch_name]['computation_config'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D
decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333

## Get global 'long_short_post_decoding' results:
curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())
(epochs_df_L, epochs_df_S), (filter_epoch_spikes_df_L, filter_epoch_spikes_df_S), (good_example_epoch_indicies_L, good_example_epoch_indicies_S), (short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset), new_all_aclus_sort_indicies, assigning_epochs_obj = PAPER_FIGURE_figure_1_add_replay_epoch_rasters(curr_active_pipeline)


In [4]:
## Use the `long_short_decoding_analyses` global result to access `long_results_obj.active_filter_epochs`:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global


In [ ]:
# curr_active_pipeline.sess.neurons.t_stop
print(list(curr_active_pipeline.sess.neurons.to_dict().keys()))

# {'t_start': self.t_start, 't_stop': self.t_stop, 'n_neurons': self.n_neurons}

# }

In [ ]:
curr_active_pipeline.sess.neurons.to_hdf('output/test_neuron_out.h5', key='neurons', session_uid='session_uid')	 # AttributeError: 'Neurons' object has no attribute '_spikes_df'... WTF?


In [10]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults
AcrossSessionsResults.build_neuron_identity_table_to_hdf('output/test_neuron_identities_out.h5', key='/', spikes_df=curr_active_pipeline.sess.spikes_df, session_uid='session_uid')


NoSuchNodeError: group ``/neuron_identities/neuron_identities`` does not have a child named ``attrs``

In [ ]:
# curr_active_pipeline.sess.neurons.spiketrains.shape # (108,)
curr_active_pipeline.sess.neurons.spiketrains[0].shape # (235,)
curr_active_pipeline.sess.neurons.spiketrains[0].dtype

In [ ]:
curr_active_pipeline.sess.flattened_spiketrains._spikes_df = add_explicit_dataframe_columns_from_lookup_df(curr_active_pipeline.sess.spikes_df, curr_active_pipeline.sess.neurons._extended_neuron_properties_df)
curr_active_pipeline.sess.spikes_df.sort_values(by=['t_seconds'], inplace=True) # Need to re-sort by timestamps once done
curr_active_pipeline.sess.spikes_df


In [ ]:
curr_active_pipeline.sess.neurons._extended_neuron_properties_df = pd.DataFrame() # add `neurons._extended_neuron_properties_df`

['aclu', 'qclu', 'cell_type', 'shank', 'cluster']

In [ ]:
import tables as tb

np.array([neuronTypesEnum[a_type.hdfcodingClassName] for a_type in curr_active_pipeline.sess.neurons.neuron_type])


In [ ]:
neuronTypesEnum['pyr']

In [ ]:
curr_active_pipeline.sess.spikes_df.n_unique

In [ ]:
cell_grouped_spikes_df = curr_active_pipeline.sess.spikes_df.groupby(['aclu'])
cell_spikes_dfs = [cell_grouped_spikes_df.get_group(a_neuron_id) for a_neuron_id in curr_active_pipeline.sess.spikes_df.spikes.neuron_ids] # a list of dataframes for each neuron_id

In [ ]:
cell_spikes_dfs[0][['shank', 'cluster', 'qclu']]



curr_active_pipeline.sess.spikes_df.spikes.extract_unique_neuron_identities()

# cell_spikes_dfs.nunique()

In [ ]:
curr_active_pipeline.sess.

In [ ]:
[cell_grouped_spikes_df.get_group(a_neuron_id)['qclu'] for a_neuron_id in curr_active_pipeline.sess.spikes_df.spikes.neuron_ids]

In [ ]:
curr_active_pipeline.sess.spikes_df.spikes.

In [ ]:


'qclu'

In [ ]:
jonathan_firing_rate_analysis_result.neuron_replay_stats_df.dtypes

In [ ]:
# Build general extended neuron info starting from `computed_neuron_info`:
computed_neuron_info_df = deepcopy(jonathan_firing_rate_analysis_result.neuron_replay_stats_df)


# computed_neuron_info['neuron_type'].to_numpy()

def get_numpy_object_array_object_type(array):
    assert len(array) > 0, f"array must have at least one element to infer its type!"
    # pd.api.types.is_array_like(array)
    # pd.api.types.is_object_dtype
    if isinstance(array, pd.Series):
        return type(array.values[0])
    elif pd.api.types.is_list_like(array):
        return type(array[0])
    else:
        raise TypeError
    
object_columns = computed_neuron_info_df.select_dtypes(include=['object']).columns
object_columns_types = [get_numpy_object_array_object_type(computed_neuron_info_df[col]) for col in object_columns] # [<enum 'SplitPartitionMembership'>, <enum 'NeuronType'>]
object_columns

In [ ]:
computed_neuron_info_df

In [ ]:
object_columns_types = [get_numpy_object_array_object_type(computed_neuron_info_df[col]) for col in object_columns]
object_columns_types

In [ ]:
object_columns_types = [get_numpy_object_array_object_type(computed_neuron_info_df[col]) for col in object_columns]

In [ ]:
type(computed_neuron_info['neuron_type'])

In [ ]:
type(computed_neuron_info['neuron_type'].values[0])

In [ ]:
# Test automatic .to_hdf for BasePositionDecoder
type(long_shared_aclus_only_decoder)

long_shared_aclus_only_decoder.to_hdf('output/automatic_test.h5', '/long_shared_aclus_only_decoder')

In [ ]:
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import main_complete_figure_generations, InstantaneousSpikeRateGroupsComputation, SingleBarResult # for `BatchSessionCompletionHandler`

_out_inst_fr_comps = InstantaneousSpikeRateGroupsComputation(instantaneous_time_bin_size_seconds=0.01) # 10ms
_out_inst_fr_comps.compute(curr_active_pipeline=curr_active_pipeline, active_context=curr_active_pipeline.sess.get_context())
LxC_ReplayDeltaMinus, LxC_ReplayDeltaPlus, SxC_ReplayDeltaMinus, SxC_ReplayDeltaPlus = _out_inst_fr_comps.LxC_ReplayDeltaMinus, _out_inst_fr_comps.LxC_ReplayDeltaPlus, _out_inst_fr_comps.SxC_ReplayDeltaMinus, _out_inst_fr_comps.SxC_ReplayDeltaPlus
LxC_ThetaDeltaMinus, LxC_ThetaDeltaPlus, SxC_ThetaDeltaMinus, SxC_ThetaDeltaPlus = _out_inst_fr_comps.LxC_ThetaDeltaMinus, _out_inst_fr_comps.LxC_ThetaDeltaPlus, _out_inst_fr_comps.SxC_ThetaDeltaMinus, _out_inst_fr_comps.SxC_ThetaDeltaPlus


In [ ]:
# Testing `SingleBarResult` serialization:`
_test_Fig2_Replay_FR: SingleBarResult = _out_inst_fr_comps.Fig2_Replay_FR[0]
_test_Fig2_Replay_FR.to_hdf('output/automatic_test.h5', '/Fig2_Replay_FR')


In [ ]:
# Testing `SpikeRateTrends` serialization
_test_LxC_ReplayDeltaMinus: SpikeRateTrends = _out_inst_fr_comps.LxC_ReplayDeltaMinus #

# _test_inst_fr_df_list: list[pd.DataFrame] = _test_LxC_ReplayDeltaMinus.inst_fr_df_list
# inst_fr_df_list: a list containing a inst_fr_df for each epoch.
# a `inst_fr_df` is a df with time bins along the rows and aclu values along the columns in the style of `unit_specific_binned_spike_counts`

# _test_inst_fr_df_list
_test_LxC_ReplayDeltaMinus.to_hdf('output/automatic_test.h5', '/LxC_ReplayDeltaMinus')

In [ ]:
_out_inst_fr_comps.to_hdf('output/automatic_test.h5', '/inst_fr_comps') # held up by SpikeRateTrends.inst_fr_df_list 

In [ ]:
## Getting IdentifyingContext as a dict of HDF5-attribute such that they could be added to a group
long_epoch_context.to_dict()


long_epoch_context.


In [ ]:
a_context: IdentifyingContext = deepcopy(long_epoch_context)


In [ ]:
a_context._get_session_context_keys()

In [ ]:
a_context.keypaths()

a_context.get_description(separator="/", include_property_names=False) # 'kdiba/gor01/one/2006-6-08_14-26-15/maze1'



In [ ]:


doc_printer = DocumentationFilePrinter(doc_output_parent_folder=Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\DEVELOPER_NOTES\DataStructureDocumentation'), doc_name='NeuropyPipeline')
doc_printer.save_documentation('NeuropyPipeline', curr_active_pipeline, non_expanded_item_keys=['stage','_reverse_cellID_index_map', 'pf_listed_colormap', 'computation_results', 'active_configs', 'logger', 'plot', '_plot_object'],
                               additional_excluded_item_classes=["<class 'pyphoplacecellanalysis.General.Pipeline.Stages.Display.Plot'>"], max_depth=16) # 'Logger'



In [ ]:
doc_printer = DocumentationFilePrinter(doc_output_parent_folder=Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\DEVELOPER_NOTES\DataStructureDocumentation'), doc_name='global_computation_results')
doc_printer.save_documentation('global_computation_results', curr_active_pipeline.global_computation_results, non_expanded_item_keys=['sess','_reverse_cellID_index_map', 'pf_listed_colormap', 'computation_results', 'active_configs', 'logger', 'plot', '_plot_object'],
                               additional_excluded_item_classes=["<class 'pyphoplacecellanalysis.General.Pipeline.Stages.Display.Plot'>"], max_depth=16) # 'Logger'



In [ ]:
doc_printer = DocumentationFilePrinter(doc_output_parent_folder=Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\DEVELOPER_NOTES\DataStructureDocumentation'), doc_name='BasePositionDecoder')
doc_printer.save_documentation('BasePositionDecoder', curr_active_pipeline.global_computation_results, non_expanded_item_keys=['sess','_reverse_cellID_index_map', 'pf_listed_colormap', 'computation_results', 'active_configs', 'logger', 'plot', '_plot_object'],
                               additional_excluded_item_classes=["<class 'pyphoplacecellanalysis.General.Pipeline.Stages.Display.Plot'>"], max_depth=16) # 'Logger'



In [ ]:
# long_one_step_decoder_1D
# type(long_one_step_decoder_1D) # pyphoplacecellanalysis.Analysis.Decoder.reconstruction.BayesianPlacemapPositionDecoder
type(long_one_step_decoder_1D.pf) # neuropy.analyses.placefields.PfND

In [ ]:
# long_short_leave_one_out_decoding_analysis.long_decoder.pf
doc_printer = DocumentationFilePrinter(doc_output_parent_folder=Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\DEVELOPER_NOTES\DataStructureDocumentation'), doc_name='BayesianPlacemapPositionDecoder')
doc_printer.save_documentation('BayesianPlacemapPositionDecoder', long_one_step_decoder_1D, non_expanded_item_keys=[], additional_excluded_item_classes=[], max_depth=16) # 'Logger'

In [ ]:
doc_printer = DocumentationFilePrinter(doc_output_parent_folder=Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\DEVELOPER_NOTES\DataStructureDocumentation'), doc_name='PfND')
doc_printer.save_documentation('PfND', long_one_step_decoder_1D.pf, non_expanded_item_keys=[], additional_excluded_item_classes=[], max_depth=16) # 'Logger'

# 2023-07-29 - *NOW* - Get the data output to `hdf5` file format for easy integration across sessions

In [ ]:
from neuropy.analyses.placefields import PfND
from neuropy.core.epoch import Epoch
from pandas.api.types import CategoricalDtype
from neuropy.core.position import Position
from neuropy.core.flattened_spiketrains import SpikesAccessor

In [ ]:
pfND: PfND = deepcopy(long_one_step_decoder_1D.pf)

In [ ]:
pfND.spikes_df.columns

['shank', 'cluster', 'aclu', 'qclu', 'cell_type', 'fragile_linear_neuron_IDX']

In [ ]:

hdf5_output_path: Path = curr_active_pipeline.get_output_path().joinpath('test_data_new.h5')
hdf5_output_path
print(f'hdf5_output_path: {hdf5_output_path}')
_pfnd_obj: PfND = long_one_step_decoder_1D.pf # self.position, self.spikes_df, self.epochs
_pfnd_obj.to_hdf(file_path=hdf5_output_path, key=key)

In [ ]:

print_object_memory_usage(jonathan_firing_rate_analysis_result) # 5.690895 MB
print_object_memory_usage(curr_long_short_post_decoding) # 0.119761 MB
print_object_memory_usage(curr_long_short_decoding_analyses) # 1383.435345 MB

In [ ]:
print_value_overview_only(jonathan_firing_rate_analysis_result)

In [ ]:
def _across_session_result_output_builder():
	""" takes the curr_active_pipeleine's loaded global results and prepares the output for use in across session aggregates.

	"""
	## Use the `long_short_decoding_analyses` global result to access `long_results_obj.active_filter_epochs`:
	curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
	long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global

	## Use the `JonathanFiringRateAnalysisResult` to get info about the long/short placefields:
	jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())
	neuron_replay_stats_df, short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset = jonathan_firing_rate_analysis_result.get_cell_track_partitions()

	# I know `neuron_replay_stats_df` is directly useful
	
	

In [ ]:
# enable_default_neptune_plots = False
enable_default_neptune_plots = True

## To file only:
with matplotlib_file_only():
	# Perform non-interactive Matplotlib operations with 'AGG' backend
	main_complete_figure_generations(curr_active_pipeline, enable_default_neptune_plots=enable_default_neptune_plots, save_figures_only=True, save_figure=True)

## Clear the Programmatically open figures:
plt.close('all') # this takes care of the matplotlib-backed figures.
curr_active_pipeline.clear_display_outputs()
curr_active_pipeline.clear_registered_output_files()



In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# Perform interactive Matplotlib operations with 'Qt5Agg' backend
_out_figures = main_complete_figure_generations(curr_active_pipeline, enable_default_neptune_plots=False, save_figures_only=False, save_figure=True)

In [ ]:
from pyphoplacecellanalysis.External.pyqtgraph import QtWidgets, QtCore, QtGui
from pyphoplacecellanalysis.GUI.Qt.MainApplicationWindows.LauncherWidget.LauncherWidget import LauncherWidget

widget = LauncherWidget()
treeWidget = widget.mainTreeWidget # QTreeWidget
widget.build_for_pipeline(curr_active_pipeline=curr_active_pipeline)
widget.show()

In [ ]:
curr_active_pipeline.display('_display_3d_interactive_spike_and_behavior_browser', 'maze') # this works now!

In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_lap_trajectories_3d
## single_combined_plot == True mode (mode 1.):
# p, laps_pages = plot_lap_trajectories_3d(curr_active_pipeline.sess, single_combined_plot=True)
# p.show()

## single_combined_plot == False mode (mode 2.):        
# p, laps_pages = plot_lap_trajectories_3d(curr_active_pipeline.sess, single_combined_plot=False, curr_num_subplots=len(curr_active_pipeline.sess.laps.lap_id), active_page_index=1)
# p.show()


In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Display import Plot

curr_active_pipeline.reload_default_display_functions()
plot_obj: Plot = curr_active_pipeline.plot

# plot_obj._display_2d_placefield_occupancy()
plot_obj._display_1d_placefield_occupancy(active_context=global_epoch_context)

In [ ]:
curr_active_pipeline.registered_display_function_docs_dict

In [ ]:
neuron_replay_stats_df = jonathan_firing_rate_analysis_result.neuron_replay_stats_df
neuron_replay_stats_df

In [ ]:
from neuropy.core.neurons import NeuronType
# Include only pyramidal aclus:
print(f'all shared_aclus: {len(shared_aclus)}\nshared_aclus: {shared_aclus}')
# shared_aclu_neuron_type = long_session.neurons.neuron_type[np.isin(long_session.neurons.neuron_ids, shared_aclus)]
shared_aclu_neuron_type = curr_active_pipeline.sess.neurons.neuron_type[np.isin(curr_active_pipeline.sess.neurons.neuron_ids, shared_aclus)]
assert len(shared_aclu_neuron_type) == len(shared_aclus)
# Find only the aclus that are pyramidal:
is_shared_aclu_pyramidal = (shared_aclu_neuron_type == NeuronType.PYRAMIDAL)
pyramidal_only_shared_aclus = shared_aclus[is_shared_aclu_pyramidal]
print(f'num pyramidal_only_shared_aclus: {len(pyramidal_only_shared_aclus)}\npyramidal_only_shared_aclus: {pyramidal_only_shared_aclus}')
# Find the other aclus (that are not pyramidal):
is_shared_aclu_NOT_pyramidal = (shared_aclu_neuron_type != NeuronType.PYRAMIDAL)
non_pyramidal_only_shared_aclus = shared_aclus[is_shared_aclu_NOT_pyramidal]
print(f'num non_pyramidal_only_shared_aclus: {len(non_pyramidal_only_shared_aclus)}\nnon_pyramidal_only_shared_aclus: {non_pyramidal_only_shared_aclus}')

# 2023-07-21 - Setup UserAnnotations class

In [ ]:
from neuropy.core.user_annotations import UserAnnotationsManager
from neuropy.utils.result_context import IdentifyingContext


annotations_man = UserAnnotationsManager()
identifying_context_dict = annotations_man.get_hardcoded_specific_session_override_dict()

relevant_entries = IdentifyingContext.matching(identifying_context_dict, criteria={'animal': 'vvp01'})
relevant_entries

In [ ]:
(epochs_df_L, epochs_df_S), (filter_epoch_spikes_df_L, filter_epoch_spikes_df_S), (good_example_epoch_indicies_L, good_example_epoch_indicies_S), (short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset), new_all_aclus_sort_indicies, assigning_epochs_obj = PAPER_FIGURE_figure_1_add_replay_epoch_rasters(curr_active_pipeline, allow_interactive_good_epochs_selection=True)

In [ ]:



user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3',display_fn_name='DecodedEpochSlices',epochs='replays',decoder='long_results_obj',user_annotation='selections')] = np.array([3, 19])
user_annotations[IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3',display_fn_name='DecodedEpochSlices',epochs='replays',decoder='short_results_obj',user_annotation='selections')] = np.array([ 5, 16, 17, 19, 20])

In [ ]:
from neuropy.utils.misc import DateTimeFormat

DateTimeFormat.WHOLE_SECONDS.now_string

In [ ]:
print(f'user annotations <good replay selections> are not found. Creating them interactively...')
user_annotations = interactive_good_epoch_selections(annotations_man=user_annotation_man, curr_active_pipeline=curr_active_pipeline)  # perform interactive selection. Should block here.
selection_idxs_L = user_annotations[selections_context_L]
selection_idxs_S = user_annotations[selections_context_S]

# Common Display Setup

In [ ]:
import matplotlib
# configure backend here
matplotlib.use('Qt5Agg')
# backend_qt5agg
# %matplotlib qt5
# %matplotlib qt
# matplotlib.use('AGG') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!

import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()
# mpl.rcParams['toolbar'] = 'None' # disable toolbars

# Figure 1) pf1D Ratemaps, Active set, etc

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_1_add_replay_epoch_rasters

pf1d_compare_graphics, (example_epoch_rasters_L, example_epoch_rasters_S), example_stacked_epoch_graphics = PAPER_FIGURE_figure_1_full(curr_active_pipeline) # did not display the pf1

In [ ]:
pf1d_compare_fig_L, pf1d_compare_fig_S = pf1d_compare_graphics.figures


In [ ]:
pf1d_compare_fig_L.show()

In [ ]:
plt.show()

In [ ]:
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import build_shared_sorted_neuronIDs

ratemap = long_pf1D.ratemap
included_unit_neuron_IDs = EITHER_subset.track_exclusive_aclus
rediculous_final_sorted_all_included_neuron_ID, rediculous_final_sorted_all_included_pfmap = build_shared_sorted_neuronIDs(ratemap, included_unit_neuron_IDs, sort_ind=new_all_aclus_sort_indicies.copy())

In [ ]:
heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(rediculous_final_sorted_all_included_pfmap, show_yticks=False, title=f"pf1D Sorted Visualization", defer_show=True)

In [ ]:
active_curves_sorted = long_pf1D.ratemap.normalized_tuning_curves[is_included][included_new_all_aclus_sort_indicies]
heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(active_curves_sorted, show_yticks=False, title=f"pf1D Sorted Visualization", defer_show=True)

In [ ]:
from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController

## Stacked Epoch Plot
example_stacked_epoch_graphics = curr_active_pipeline.display('_display_long_and_short_stacked_epoch_slices', defer_render=False, save_figure=True)
pagination_controller_L, pagination_controller_S = example_stacked_epoch_graphics.plot_data['controllers']
ax_L, ax_S = example_stacked_epoch_graphics.axes
final_figure_context_L, final_context_S = example_stacked_epoch_graphics.context

In [ ]:
from pyphoplacecellanalysis.GUI.Qt.Mixins.PaginationMixins import SelectionsObject
from neuropy.core.user_annotations import UserAnnotationsManager

user_annotations = UserAnnotationsManager.get_user_annotations()
user_annotations

In [ ]:
## Capture current user selection
saved_selection_L: SelectionsObject = pagination_controller_L.save_selection()
saved_selection_S: SelectionsObject = pagination_controller_S.save_selection()
user_annotations_L_context = saved_selection_L.figure_ctx.adding_context_if_missing(user_annotation='selections')
user_annotations_S_context = saved_selection_S.figure_ctx.adding_context_if_missing(user_annotation='selections')
user_annotations[user_annotations_L_context] = saved_selection_L.flat_all_data_indicies[saved_selection_L.is_selected]
user_annotations[user_annotations_S_context] = saved_selection_S.flat_all_data_indicies[saved_selection_S.is_selected]
# Updates the context. Needs to generate the code.

## Generate code to insert int user_annotations:
print('Add the following code to UserAnnotationsManager.get_user_annotations() function body:')
print(f"\t\tuser_annotations[{user_annotations_L_context.get_initialization_code_string()}] = np.array({list(saved_selection_L.flat_all_data_indicies[saved_selection_L.is_selected])})")
print(f"\t\tuser_annotations[{user_annotations_S_context.get_initialization_code_string()}] = np.array({list(saved_selection_S.flat_all_data_indicies[saved_selection_S.is_selected])})")


In [ ]:
## Update the currently displayed selections with the user annotations:


## Capture current user selection
saved_selection_L = pagination_controller_L.save_selection()
saved_selection_S = pagination_controller_S.save_selection()

saved_selection_L = saved_selection_L.update_selections_from_annotations(user_annotations_dict=user_annotations)
saved_selection_S = saved_selection_S.update_selections_from_annotations(user_annotations_dict=user_annotations)
## re-apply the selections:
pagination_controller_L.restore_selections(saved_selection_L)
pagination_controller_S.restore_selections(saved_selection_S)


In [ ]:
# unit_colors_list = None # default rainbow of colors for the raster plots
neuron_qcolors_list = [pg.mkColor('black') for aclu in EITHER_subset.track_exclusive_aclus] # solid green for all
unit_colors_list = DataSeriesColorHelpers.qColorsList_to_NDarray(neuron_qcolors_list, is_255_array=True)

# Copy and modify the colors for the cells that are long/short exclusive:
unit_colors_list_L = deepcopy(unit_colors_list)
is_L_exclusive = np.isin(EITHER_subset.track_exclusive_aclus, long_exclusive.track_exclusive_aclus) # get long exclusive
unit_colors_list_L[0, is_L_exclusive] = 255 # [1.0, 0.0, 0.0, 1.0]
unit_colors_list_L[1, is_L_exclusive] = 0.0
unit_colors_list_L[2, is_L_exclusive] = 0.0

unit_colors_list_S = deepcopy(unit_colors_list)
is_S_exclusive = np.isin(EITHER_subset.track_exclusive_aclus, short_exclusive.track_exclusive_aclus) # get short exclusive
unit_colors_list_S[0, is_S_exclusive] = 0.0 # [1.0, 0.0, 0.0, 1.0]
unit_colors_list_S[1, is_S_exclusive] = 0.0
unit_colors_list_S[2, is_S_exclusive] = 255.0


In [ ]:

spikes_df = an_epoch_spikes_df.copy()
scatterplot_tooltips_kwargs = Render2DScrollWindowPlotMixin._build_spike_data_tuples_from_spikes_df(spikes_df)
override_scatter_plot_kwargs = {**scatterplot_tooltips_kwargs}

In [ ]:
override_scatter_plot_kwargs = {} 

In [ ]:

vtick = _build_default_tick(tick_width=1.0)

# pxMode: If True, spots are always the same size regardless of scaling, and size is given in px. Otherwise, size is in scene coordinates and the spots scale with the view. To ensure effective caching, QPen and QBrush objects should be reused as much as possible. Default is True

# size: The size (or list of sizes) of spots. If pxMode is True, this value is in pixels. Otherwise, it is in the item’s local coordinate system.

override_scatter_plot_kwargs = dict(pxMode=True, symbol=vtick, size=10, pen={'color': 'w', 'width': 1.0})

# override_scatter_plot_kwargs = dict(pxMode=False, symbol=vtick, size=1, pen={'color': 'w', 'width': 1.0})


In [ ]:
# Set the symbol properties
# symbol = pg.mkPen('w', width=1)  # Pen for the lines
size = 1.0  # Fixed x-width
symbol_brush = None  # No brush for the symbol (transparent fill)

# override_scatter_plot_kwargs = dict(pxMode=False, symbol='|', size=size, pen={'color': 'w', 'width': 1.0}) # , brush=symbol_brush

override_scatter_plot_kwargs = dict(pxMode=False, symbol='arrow_up', size=1.0, pen={'color': 'w', 'width': 1.0}, hoverable=True) # 

In [ ]:
filter_epoch_spikes_df_L.spikes.rebuild_fragile_linear_neuron_IDXs();
filter_epoch_spikes_df_S.spikes.rebuild_fragile_linear_neuron_IDXs();

In [ ]:
app_L, win_L, plots_L, plots_data_L = plot_multiple_raster_plot(epochs_df_L, filter_epoch_spikes_df_L, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs,
										epoch_id_key_name='replay_epoch_id', scatter_app_name="Long Decoded Example Replays")

In [ ]:
app_S, win_S, plots_S, plots_data_S = plot_multiple_raster_plot(epochs_df_S, filter_epoch_spikes_df_S, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs,
                                                                 epoch_id_key_name='replay_epoch_id', scatter_app_name="Short Decoded Example Replays")

In [ ]:
# Test single `plot_raster_plot` calls
an_epoch = list(epochs_df_L.itertuples())[0]
an_epoch_spikes_df = filter_epoch_spikes_df_L[filter_epoch_spikes_df_L['replay_epoch_id'] == an_epoch.Index]

_out_single_raster_plot = plot_raster_plot(an_epoch_spikes_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=None, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs, scatter_app_name="test1")
_out_single_raster_plot2 = plot_raster_plot(an_epoch_spikes_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs, scatter_app_name="test2")

In [ ]:
app_alt, win_alt, plots_alt, plots_data_alt = plot_multiple_raster_plot(epochs_df_L, filter_epoch_spikes_df_L, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=None, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs,
                                                                         epoch_id_key_name='replay_epoch_id', scatter_app_name="ALT Long Decoded Example Replays")

### Testing `plot_kourosh_activity_style_figure` for debugging:

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import plot_kourosh_activity_style_figure
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.helpers import _helper_make_scatterplot_clickable

plot_aclus = EITHER_subset.track_exclusive_aclus.copy()
# plot_aclus = EITHER_subset.track_exclusive_aclus[new_all_aclus_sort_indicies].copy()
_out_A = plot_kourosh_activity_style_figure(long_results_obj, long_session, plot_aclus, unit_sort_order=new_all_aclus_sort_indicies, epoch_idx=13, callout_epoch_IDXs=None, skip_rendering_callouts=False)

In [ ]:
app, win, plots, plots_data = _out_A
# plots

In [ ]:
#TODO 2023-06-27 10:42: - [ ] Desperitely need a class that "explodes" the important variables and their types out of a DynamicParameters (dict-like) or other object.


In [ ]:
_out_n = plot_kourosh_activity_style_figure(long_results_obj, long_session, EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, epoch_idx=49, callout_epoch_IDXs=np.arange(6), skip_rendering_callouts=False)

# 2023-07-14 - LxC and SxC PhoJonathanSession plots

In [ ]:
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import BatchPhoJonathanFiguresHelper

## Get global 'jonathan_firing_rate_analysis' results:
curr_jonathan_firing_rate_analysis = curr_active_pipeline.global_computation_results.computed_data['jonathan_firing_rate_analysis']
neuron_replay_stats_df, rdf, aclu_to_idx, irdf = curr_jonathan_firing_rate_analysis['neuron_replay_stats_df'], curr_jonathan_firing_rate_analysis['rdf']['rdf'], curr_jonathan_firing_rate_analysis['rdf']['aclu_to_idx'], curr_jonathan_firing_rate_analysis['irdf']['irdf']

# with VizTracer(output_file=f"viztracer_display_BatchPhoJonathanFiguresHelper_PlusPDF_20.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
active_out_figures_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df, included_unit_neuron_IDs=XOR_subset.track_exclusive_aclus, n_max_page_rows=20, write_vector_format=False, write_png=True) # active_out_figures_dict: {IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'long_only', '(12,21,48)')>: <Figure size 1920x660 with 12 Axes>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'short_only', '(18,19,65)')>: <Figure size 1920x660 with 12 Axes>}
print(f'active_out_figures_dict: {active_out_figures_dict}')


In [ ]:
# curr_active_pipeline.display(display_function='_display_2d_placefield_result_plot_ratemaps_2D', active_session_configuration_context=long_epoch_context)


# curr_active_pipeline.display(display_function='_display_placemaps_pyqtplot_2D', active_session_configuration_context=long_epoch_context)

curr_active_pipeline.display(display_function='_display_2d_placefield_occupancy', active_session_configuration_context=long_epoch_context)




In [ ]:
fig = plt.figure() # new figure to hold the result
# can show the figures by looping through and calling
for a_ctxt, a_fig in active_out_figures_dict.items():
    print(f'showing: {a_ctxt}')
    a_fig.show()
    


In [ ]:

curr_active_pipeline.registered_output_files_list

# Figure 2) Firing Rate Bar Graphs

In [ ]:
# Instantaneous versions:
from pyphocorehelpers.mixins.serialized import SerializedAttributesAllowBlockSpecifyingClass
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PaperFigureTwo, InstantaneousSpikeRateGroupsComputation
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends

_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=0.01) # 10ms
_out_fig_2.compute(curr_active_pipeline=curr_active_pipeline, active_context=curr_active_pipeline.sess.get_context())

_out_fig_2_figs = {}

restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# Perform interactive Matplotlib operations with 'Qt5Agg' backend
_out_fig_2_figs = _out_fig_2.display(defer_show=True, save_figure=True, active_context=curr_active_pipeline.sess.get_context())

## Extract for debugging/checking:
_out_inst_fr_comps = _out_fig_2.computation_result
LxC_ReplayDeltaMinus, LxC_ReplayDeltaPlus, SxC_ReplayDeltaMinus, SxC_ReplayDeltaPlus = _out_inst_fr_comps.LxC_ReplayDeltaMinus, _out_inst_fr_comps.LxC_ReplayDeltaPlus, _out_inst_fr_comps.SxC_ReplayDeltaMinus, _out_inst_fr_comps.SxC_ReplayDeltaPlus
LxC_ThetaDeltaMinus, LxC_ThetaDeltaPlus, SxC_ThetaDeltaMinus, SxC_ThetaDeltaPlus = _out_inst_fr_comps.LxC_ThetaDeltaMinus, _out_inst_fr_comps.LxC_ThetaDeltaPlus, _out_inst_fr_comps.SxC_ThetaDeltaMinus, _out_inst_fr_comps.SxC_ThetaDeltaPlus
# LxC_ThetaDeltaMinus # cell_agg_inst_fr_list


In [ ]:
_out_inst_fr_comps = InstantaneousSpikeRateGroupsComputation(instantaneous_time_bin_size_seconds=0.01) # 10ms
_out_inst_fr_comps.compute(curr_active_pipeline=curr_active_pipeline, active_context=curr_active_pipeline.sess.get_context())
LxC_ReplayDeltaMinus, LxC_ReplayDeltaPlus, SxC_ReplayDeltaMinus, SxC_ReplayDeltaPlus = _out_inst_fr_comps.LxC_ReplayDeltaMinus, _out_inst_fr_comps.LxC_ReplayDeltaPlus, _out_inst_fr_comps.SxC_ReplayDeltaMinus, _out_inst_fr_comps.SxC_ReplayDeltaPlus
LxC_ThetaDeltaMinus, LxC_ThetaDeltaPlus, SxC_ThetaDeltaMinus, SxC_ThetaDeltaPlus = _out_inst_fr_comps.LxC_ThetaDeltaMinus, _out_inst_fr_comps.LxC_ThetaDeltaPlus, _out_inst_fr_comps.SxC_ThetaDeltaMinus, _out_inst_fr_comps.SxC_ThetaDeltaPlus

In [ ]:
inst_fr_output_filename = 'long_short_inst_firing_rates.pkl'
inst_fr_output_path = curr_active_pipeline.get_output_path().joinpath(inst_fr_output_filename).resolve()
inst_fr_output_path

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData

# saveData(inst_fr_output_path, _out_fig_2)
saveData(inst_fr_output_path, _out_fig_2.to_dict())


In [ ]:
from pyphocorehelpers.print_helpers import print_filesystem_file_size, print_object_memory_usage


print_object_memory_usage(_out_fig_2) #  2743.397699 MB
# print_object_memory_usage(_out_fig_2.to_dict()) #  2743.397699 MB
# print_object_memory_usage(_out_fig_2._pipeline_file_callback_fn) #  2737.983306 MB
print(print_object_memory_usage(_out_inst_fr_comps)) #  5.489502 MB


In [ ]:

print_object_memory_usage((LxC_ReplayDeltaMinus, LxC_ReplayDeltaPlus, SxC_ReplayDeltaMinus, SxC_ReplayDeltaPlus, LxC_ThetaDeltaMinus, LxC_ThetaDeltaPlus, SxC_ThetaDeltaMinus, SxC_ThetaDeltaPlus))

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends


srt1 = PaperFigureTwo(**loadData(inst_fr_output_path))


In [ ]:
srt2 = loadData(inst_fr_output_path)

# When combining two `SpikeRateTrends` objects, they may differ in any/all (n_timebins, n_epochs, n_cells) making it difficult to safely concatenate them. 
# `LxC_ThetaDeltaMinus`
# srt1: SpikeRateTrends = deepcopy(LxC_ThetaDeltaMinus) # SpikeRateTrends(...)
# srt2: SpikeRateTrends = deepcopy(LxC_ThetaDeltaMinus) # SpikeRateTrends(...)
# concatenated = srt1 + srt2
# concatenated = srt1.concatenate(srt2)
concatenated = SpikeRateTrends.concatenate(srt1, srt2)


In [ ]:
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]] # only uses global_session
(epochs_df_L, epochs_df_S), (filter_epoch_spikes_df_L, filter_epoch_spikes_df_S), (good_example_epoch_indicies_L, good_example_epoch_indicies_S), (short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset), new_all_aclus_sort_indicies, assigning_epochs_obj = PAPER_FIGURE_figure_1_add_replay_epoch_rasters(curr_active_pipeline)

long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
long_laps, long_replays, short_laps, short_replays, global_laps, global_replays = [long_short_fr_indicies_analysis_results[k] for k in ['long_laps', 'long_replays', 'short_laps', 'short_replays', 'global_laps', 'global_replays']]


In [ ]:
jonathan_firing_rate_analysis_result.neuron_replay_stats_df

In [ ]:
long_exclusive.track_exclusive_df

In [ ]:
long_exclusive.track_exclusive_df['lap_delta_minus_inst_fr'] = LxC_ThetaDeltaMinus.cell_agg_inst_fr_list # (n_cells, )
long_exclusive.track_exclusive_df['lap_delta_plus_inst_fr'] = LxC_ThetaDeltaPlus.cell_agg_inst_fr_list # (n_cells, )
long_exclusive.track_exclusive_df


In [ ]:
# d = LxC_ThetaDeltaMinus.epoch_agg_inst_fr_list # (n_epochs, n_cells)
# d = d[:,1]
d = LxC_ThetaDeltaMinus.cell_agg_inst_fr_list # (n_cells, )
d
long_exclusive.track_exclusive_df['lap_delta_minus_inst_fr'] = LxC_ThetaDeltaMinus.cell_agg_inst_fr_list # (n_cells, )


In [ ]:
## Drop out the very low (inactive) Epochs... I guess? But we want it to be influenced by the inactive epochs.
curr_active_pipeline.	

In [ ]:

len(LxC_ThetaDeltaMinus.inst_fr_signals_list) # n_epochs
d = LxC_ThetaDeltaMinus.inst_fr_df_list
d[0]
# (n_epochs, n_cells)
# print(f'{d.shape}')

In [ ]:
LxC_ThetaDeltaPlus
# Look at percent change
(LxC_ThetaDeltaPlus.cell_agg_inst_fr_list - LxC_ThetaDeltaMinus.cell_agg_inst_fr_list) / LxC_ThetaDeltaMinus.cell_agg_inst_fr_list

## NOW: 2023-07-11 - Testing Batch-computed inst_firing_rates

In [ ]:
from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PaperFigureTwo, InstantaneousSpikeRateGroupsComputation
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations


## Load the saved across-session results:
inst_fr_output_filename = 'long_short_inst_firing_rate_result_handlers_2023-07-12.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

In [ ]:
## Aggregate across all of the sessions to build a new combined `InstantaneousSpikeRateGroupsComputation`, which can be used to plot the "PaperFigureTwo", bar plots for many sessions.
global_multi_session_context = IdentifyingContext(format_name='kdiba', num_sessions=num_sessions) # some global context across all of the sessions, not sure what to put here.

# To correctly aggregate results across sessions, it only makes sense to combine entries at the `.cell_agg_inst_fr_list` variable and lower (as the number of cells can be added across sessions, treated as unique for each session).

## Display the aggregate across sessions:
_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=0.01) # WARNING: we didn't save this info
_out_fig_2.computation_result = across_session_inst_fr_computation
_out_fig_2.active_identifying_session_ctx = across_session_inst_fr_computation.active_identifying_session_ctx
# Set callback, the only self-specific property
_out_fig_2._pipeline_file_callback_fn = curr_active_pipeline.output_figure # lambda args, kwargs: self.write_to_file(args, kwargs, curr_active_pipeline)

In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# Perform interactive Matplotlib operations with 'Qt5Agg' backend
_fig_2_theta_out, _fig_2_replay_out = _out_fig_2.display(active_context=global_multi_session_context, title_modifier_fn=lambda original_title: f"{original_title} ({num_sessions} sessions)", save_figure=True)
	
_out_fig_2.perform_save()

# Figure 3) `PAPER_FIGURE_figure_3`: Firing Rate Index and Long/Short Firing Rate Replays v. Laps

In [ ]:
from neuropy.utils.matplotlib_helpers import FormattedFigureText
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _plot_long_short_firing_rate_indicies
curr_active_pipeline.reload_default_display_functions()

_out, _out2 = PAPER_FIGURE_figure_3(curr_active_pipeline, defer_render=False, save_figure=True)

In [ ]:
_out2.figures
_out.axes

In [ ]:
1.0-0.090

In [ ]:
fig = _out.figures
fig
# Computed long ($L$)|short($S$) firing rate indicies

# 2023-07-07 - `batch_extended_programmatic_figures` Testing

In [ ]:
curr_active_pipeline.reload_default_display_functions()

neptuner = batch_perform_all_plots(curr_active_pipeline, enable_neptune=True)

In [ ]:

batch_extended_programmatic_figures(curr_active_pipeline, write_vector_format=False, write_png=False, debug_print=False)

# Interactive Selection of User Annotations:

In [ ]:
from neuropy.core.user_annotations import UserAnnotationsManager
from pyphoplacecellanalysis.GUI.Qt.Mixins.PaginationMixins import SelectionsObject
from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController

user_annotation_man = UserAnnotationsManager()
user_annotations = user_annotation_man.get_user_annotations()

final_context_L = curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='long_results_obj')
final_context_S = curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='short_results_obj')
# _out_pagination_controller.params.active_identifying_figure_ctx.adding_context(None,  user_annotation="selections")
selections_context_L = final_context_L.adding_context(None,  user_annotation="selections")
selections_context_S = final_context_S.adding_context(None,  user_annotation="selections")


In [ ]:
# recover it from the last_added_display_output
example_stacked_epoch_graphics = curr_active_pipeline.last_added_display_output # recover it from the last_added_display_output
pagination_controller_L, pagination_controller_S = example_stacked_epoch_graphics.plot_data['controllers']
ax_L, ax_S = example_stacked_epoch_graphics.axes
figure_context_L, figure_context_S = example_stacked_epoch_graphics.context

In [ ]:
# user_annotations = UserAnnotationsManager.get_user_annotations()
# user_annotations = self.get_user_annotations()

## Capture current user selection

# def _update_user_annotations_from_selections():
# 	""" captures:
# 		user_annotations
# 		pagination_controller_L, pagination_controller_S
# 	"""
saved_selection_L: SelectionsObject = pagination_controller_L.save_selection()
user_annotations_L_context: IdentifyingContext = saved_selection_L.figure_ctx.adding_context_if_missing(user_annotation='selections')
print(f'{user_annotations_L_context}\n\t{saved_selection_L.flat_all_data_indicies[saved_selection_L.is_selected]}')
user_annotations[user_annotations_L_context] = saved_selection_L.flat_all_data_indicies[saved_selection_L.is_selected]

saved_selection_S: SelectionsObject = pagination_controller_S.save_selection()
user_annotations_S_context: IdentifyingContext = saved_selection_S.figure_ctx.adding_context_if_missing(user_annotation='selections')
print(f'{user_annotations_S_context}\n\t{saved_selection_S.flat_all_data_indicies[saved_selection_S.is_selected]}')
user_annotations[user_annotations_S_context] = saved_selection_S.flat_all_data_indicies[saved_selection_S.is_selected]

## Generate code to insert int user_annotations:
print('Add the following code to `pyphoplacecellanalysis.General.Model.user_annotations.UserAnnotationsManager.get_user_annotations()` function body:')
print(f"user_annotations[{user_annotations_L_context.get_initialization_code_string()}] = np.array({list(saved_selection_L.flat_all_data_indicies[saved_selection_L.is_selected])})")
print(f"user_annotations[{user_annotations_S_context.get_initialization_code_string()}] = np.array({list(saved_selection_S.flat_all_data_indicies[saved_selection_S.is_selected])})")

# Stopping at (128, 89)

# Updates the context. Needs to generate the code.
# _update_user_annotations_from_selections()


In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

## Stacked Epoch Plot
example_stacked_epoch_graphics = curr_active_pipeline.display('_display_long_and_short_stacked_epoch_slices', defer_render=False, save_figure=False) # Build a new one
# example_stacked_epoch_graphics = curr_active_pipeline.last_added_display_output # recover it from the last_added_display_output
pagination_controller_L, pagination_controller_S = example_stacked_epoch_graphics.plot_data['controllers'] # DecodedEpochSlicesPaginatedFigureController, DecodedEpochSlicesPaginatedFigureController
ax_L, ax_S = example_stacked_epoch_graphics.axes
figure_context_L, figure_context_S = example_stacked_epoch_graphics.context

In [ ]:
## After launching the interactive Stacked Epoch Plots for user epoch selection, try to update the selection with previously added annotations:
saved_selection_L: SelectionsObject = pagination_controller_L.save_selection()
saved_selection_S: SelectionsObject = pagination_controller_S.save_selection()
saved_selection_L = user_annotation_man.update_selections_from_annotations(saved_selection_L, user_annotations)
saved_selection_S = user_annotation_man.update_selections_from_annotations(saved_selection_S, user_annotations)

## re-apply the selections:
pagination_controller_L.restore_selections(saved_selection_L, defer_render=True)
pagination_controller_S.restore_selections(saved_selection_S, defer_render=True)

In [ ]:
ax_L[0].figure.canvas.draw()  # .figures.canvas.draw()

In [ ]:
ax_S[0].figure.canvas.draw()  # .figures.canvas.draw()

In [ ]:
## Capture current user selection:
## Capture current user selection

# def _update_user_annotations_from_selections():
# 	""" captures:
# 		user_annotations
# 		pagination_controller_L, pagination_controller_S
# 	"""
saved_selection_L: SelectionsObject = pagination_controller_L.save_selection()
user_annotations_L_context: IdentifyingContext = saved_selection_L.figure_ctx.adding_context_if_missing(user_annotation='selections')
print(f'{user_annotations_L_context}\n\t{saved_selection_L.flat_all_data_indicies[saved_selection_L.is_selected]}')
user_annotations[user_annotations_L_context] = saved_selection_L.flat_all_data_indicies[saved_selection_L.is_selected]

saved_selection_S: SelectionsObject = pagination_controller_S.save_selection()
user_annotations_S_context: IdentifyingContext = saved_selection_S.figure_ctx.adding_context_if_missing(user_annotation='selections')
print(f'{user_annotations_S_context}\n\t{saved_selection_S.flat_all_data_indicies[saved_selection_S.is_selected]}')
user_annotations[user_annotations_S_context] = saved_selection_S.flat_all_data_indicies[saved_selection_S.is_selected]

## Generate code to insert int user_annotations:
print('Add the following code to `pyphoplacecellanalysis.General.Model.user_annotations.UserAnnotationsManager.get_user_annotations()` function body:')
print(f"user_annotations[{user_annotations_L_context.get_initialization_code_string()}] = np.array({list(saved_selection_L.flat_all_data_indicies[saved_selection_L.is_selected])})")
print(f"user_annotations[{user_annotations_S_context.get_initialization_code_string()}] = np.array({list(saved_selection_S.flat_all_data_indicies[saved_selection_S.is_selected])})")



In [ ]:
from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import interactive_good_epoch_selections

## try to get the user annotations for this session:
try:
	selection_idxs_L = user_annotations[selections_context_L]
	selection_idxs_S = user_annotations[selections_context_S]
except KeyError as e:
	print(f'user annotations <good replay selections> are not found. Creating them interactively...')
	# user_annotation_man.interactive_good_epoch_selections(curr_active_pipeline=curr_active_pipeline) # perform interactive selection. Should block here.
	# user_annotations = interactive_good_epoch_selections(annotations_man=user_annotation_man, curr_active_pipeline=curr_active_pipeline) # perform interactive selection. Should block here.

	# Showing
	restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

	## Stacked Epoch Plot
	example_stacked_epoch_graphics = curr_active_pipeline.display('_display_long_and_short_stacked_epoch_slices', defer_render=False, save_figure=False)
	pagination_controller_L, pagination_controller_S = example_stacked_epoch_graphics.plot_data['controllers']
	ax_L, ax_S = example_stacked_epoch_graphics.axes
	figure_context_L, figure_context_S = example_stacked_epoch_graphics.context

except Exception as e:
	print('Unhandled exception: {e}')
	raise

In [ ]:
plt.show()

In [ ]:

user_annotations_L_context.get_initialization_code_string(subset_includelist=['format_name','animal','exper_name', 'session_name'])


In [ ]:
from neuropy.utils.result_context import IdentifyingContext

IdentifyingContext._get_session_context_keys()

# user_annotations_L_context.get_subset(subset_includelist=[IdentifyingContext._get_session_context_keys()])

In [ ]:
selection_idxs_L = user_annotations[selections_context_L]
selection_idxs_S = user_annotations[selections_context_S]

# 2023-07-19 - Validation with 3D tools

In [ ]:
# curr_active_pipeline.plot.

display_output = {}
active_config_name = long_epoch_name

# 3D Plotters

## 🪟 ipcDataExplorer - 3D Interactive Tuning Curves Plotter

In [ ]:
pActiveTuningCurvesPlotter = None

zScalingFactor = 2000.0 # worked well before with default params
# zScalingFactor = 50.0 # worked well before with default params
display_output = display_output | curr_active_pipeline.display('_display_3d_interactive_tuning_curves_plotter', active_config_name, extant_plotter=display_output.get('pActiveTuningCurvesPlotter', None), panel_controls_mode='Qt', should_nan_non_visited_elements=False, zScalingFactor=zScalingFactor, separate_window=False)
ipcDataExplorer = display_output['ipcDataExplorer']
display_output['pActiveTuningCurvesPlotter'] = display_output.pop('plotter') # rename the key from the generic "plotter" to "pActiveSpikesBehaviorPlotter" to avoid collisions with others
pActiveTuningCurvesPlotter = display_output['pActiveTuningCurvesPlotter']
root_dockAreaWindow, placefieldControlsContainerWidget, pf_widgets = display_output['pane'] # for Qt mode:

## 🪟 ipspikesDataExplorer - 3D Interactive Spike and Behavior Plotter

In [ ]:
pActiveSpikesBehaviorPlotter = None
display_output = display_output | curr_active_pipeline.display('_display_3d_interactive_spike_and_behavior_browser', active_config_name, extant_plotter=display_output.get('pActiveSpikesBehaviorPlotter', None)) # Works now!
ipspikesDataExplorer = display_output['ipspikesDataExplorer']
display_output['pActiveSpikesBehaviorPlotter'] = display_output.pop('plotter') # rename the key from the generic "plotter" to "pActiveSpikesBehaviorPlotter" to avoid collisions with others
pActiveSpikesBehaviorPlotter = display_output['pActiveSpikesBehaviorPlotter']

### ✅ Test adding the nearest predicted/decoded position as a red point to the 3D plotter:

In [ ]:
_debug_print = False

def _update_nearest_decoded_most_likely_position_callback(start_t, end_t):
    """ Only uses end_t
    Implicitly captures: ipspikesDataExplorer, _get_nearest_decoded_most_likely_position_callback
    
    Usage:
        _update_nearest_decoded_most_likely_position_callback(0.0, ipspikesDataExplorer.t[0])
        _conn = ipspikesDataExplorer.sigOnUpdateMeshes.connect(_update_nearest_decoded_most_likely_position_callback)

    """
    def _get_nearest_decoded_most_likely_position_callback(t):
        """ A callback that when passed a visualization timestamp (the current time to render) returns the most likely predicted position provided by the active_two_step_decoder
        Implicitly captures:
            active_one_step_decoder, active_two_step_decoder
        Usage:
            _get_nearest_decoded_most_likely_position_callback(9000.1)
        """
        active_time_window_variable = active_one_step_decoder.time_window_centers # get time window centers (n_time_window_centers,) # (4060,)
        active_most_likely_positions = active_one_step_decoder.most_likely_positions.T # (4060, 2) NOTE: the most_likely_positions for the active_one_step_decoder are tranposed compared to the active_two_step_decoder
        # active_most_likely_positions = active_two_step_decoder.most_likely_positions # (2, 4060)
        assert np.shape(active_time_window_variable)[0] == np.shape(active_most_likely_positions)[1], f"timestamps and num positions must be the same but np.shape(active_time_window_variable): {np.shape(active_time_window_variable)} and np.shape(active_most_likely_positions): {np.shape(active_most_likely_positions)}!"
        last_window_index = np.searchsorted(active_time_window_variable, t, side='left') # side='left' ensures that no future values (later than 't') are ever returned
        # TODO: CORRECTNESS: why is it returning an index that corresponds to a time later than the current time?
        # for current time t=9000.0
        #     last_window_index: 1577
        #     last_window_time: 9000.5023
        # EH: close enough
        last_window_time = active_time_window_variable[last_window_index] # If there is no suitable index, return either 0 or N (where N is the length of `a`).
        displayed_time_offset = t - last_window_time # negative value if the window time being displayed is in the future
        if _debug_print:
            print(f'for current time t={t}\n\tlast_window_index: {last_window_index}\n\tlast_window_time: {last_window_time}\n\tdisplayed_time_offset: {displayed_time_offset}')
        return (last_window_time, *list(np.squeeze(active_most_likely_positions[:, last_window_index]).copy()))

    t = end_t # the t under consideration should always be the end_t. This is written this way just for compatibility with the ipspikesDataExplorer.sigOnUpdateMeshes (float, float) signature
    curr_t, curr_x, curr_y = _get_nearest_decoded_most_likely_position_callback(t)
    curr_debug_point = [curr_x, curr_y, ipspikesDataExplorer.z_fixed[-1]]
    if _debug_print:
        print(f'tcurr_debug_point: {curr_debug_point}') # \n\tlast_window_time: {last_window_time}\n\tdisplayed_time_offset: {displayed_time_offset}
    ipspikesDataExplorer.perform_plot_location_point('debug_point_plot', curr_debug_point, color='r', render=True)
    return curr_debug_point

_update_nearest_decoded_most_likely_position_callback(0.0, ipspikesDataExplorer.t[0])
# _conn = pg.SignalProxy(ipspikesDataExplorer.sigOnUpdateMeshes, rateLimit=14, slot=_update_nearest_decoded_most_likely_position_callback)
_conn = ipspikesDataExplorer.sigOnUpdateMeshes.connect(_update_nearest_decoded_most_likely_position_callback)